このノートブックでは、やや複雑なサンプルを作成することにします。1つの正規のツール（検索）と、99の偽ツール（無意味なもの）を用意することにしましょう。そして、ユーザーからの入力を受けて、クエリに関連するツールを検索するステップをプロンプトテンプレートに追加します。

## 環境を整える

必要なインポートなどを行う。

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## ツールのセットアップ

正規のツール（検索）を1つ、そして99個の偽ツールを作成します。

In [ ]:
# エージェントがユーザーの問い合わせに答えるために使用できるツールを定義する。
search = SerpAPIWrapper()
search_tool = Tool(
        name = "Search",
        func=search.run,
        description="時事問題への回答が必要なときに便利です。"
    )
def fake_func(inp: str) -> str:
    return "foo"
fake_tools = [
    Tool(
        name=f"foo-{i}", 
        func=fake_func, 
        description=f"数値 {i} に関する情報を得るために使える馬鹿げた機能"
    ) 
    for i in range(99)
]
ALL_TOOLS = [search_tool] + fake_tools

## ツールリトリーバー

vectorstoreを使って、各ツールの説明文の埋め込み（ベクトル）を作成することにします。そして、入力されたクエリに対して埋め込み（ベクトル）を作成し、関連するツールの類似性検索を行うことができます。

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [ ]:
docs = [Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(ALL_TOOLS)]

In [ ]:
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [ ]:
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

これで、この検索がうまくいきそうなのかどうか、テストができます。

In [ ]:
get_tools("天気はどうですか？")

In [ ]:
get_tools("番号13は何ですか？")

## プロンプトテンプレート

プロンプトテンプレートはかなり標準的です。なぜなら、実際のプロンプトテンプレートでは、ロジックの変更はそれほど多くはなく、検索方法が変わっているだけだからです。

In [ ]:
# ベースとなるテンプレートを設定する
template = """Answer the following questions as best you can, Be sure to answer in Japanese. You have access to the following tools::

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

カスタムプロンプトテンプレートにtools_getterの概念が追加され、使用するツールを選択するため、入力時に呼び出すようになりました。

In [ ]:
from typing import Callable
# プロンプトテンプレートを設定する
class CustomPromptTemplate(StringPromptTemplate):
    # 使用するテンプレート
    template: str
    ############## NEW ######################
    # 使用できるツールの一覧
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # 中間ステップ（AgentAction, Observationタプル）を取得する。
        # 特定の方法でフォーマットする
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # 変数agent_scratchpadにその値を設定します。
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # 用意されたツール一覧からtools変数を作成する。
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # 用意されたツールの名称リストを作る
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # `agent_scratchpad`、`tools`、`tool_names`は動的に生成されるため、省略されています。
    # これは、`intermediate_steps`変数が必要だからです。
    input_variables=["input", "intermediate_steps"]
)

## 出力パーサー

出力フォーマットについて何も変更していないため、出力パーサーは前のノートブックから変更されていません。

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## LLM、ストップシーケンス、およびエージェントの設定

これも前のノートブックと同じです。

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# LLMとプロンプトで構成されるLLMチェーン
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tools = get_tools("天気はどうですか")
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

## エージェントを使う

それでは使ってみましょう！

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("浜松市の天気はどうですか？")